During this week’s Independent project, you will get to test the skills that you learned this week. More specifically, you will get the test your understanding of the following learning outcomes.

**Overall Learning Outcomes**

I can understand and apply supervised learning algorithms such as regression, decision trees, KNN, SVM, naive Bayes, random forests to solving business problems.
I can understand the benefits, limitations, and requirements of various supervised learning algorithms.

# Experimental Procedure:


* Download the two datasets from the given links:

* Randomly partition each dataset into two parts i.e 80 - 20  sets.

* For dataset 2, perform classification of the testing set samples using the Naive Bayes Classifier.

* Compute the accuracy (percentage of correct classification).

* Report the confusion matrix of each classifier.

* Repeat step 2 to step 4 twice, each time splitting the datasets differently i.e. 70-30, 60-40, then note the outcomes of your modeling.

* Suggest and apply at least one of the optimization techniques that you learned earlier this week.

* Provide further recommendations to improve both classifiers.

1. Defining the Question

* a) Specifying the Data Analytic Question



* b) Defining the Metric for Success

Having a model with a >70% accuracy rate

* c) Understanding the context

Using a naive bayers model to predict our feature variable

* d) Recording the Experimental Design

We will use the establish process learnt in moringa for analysis and documenting

* e) Data Relevance

The data is a common dataset used for learning ML techniques.Used for Classifying Email as Spam or Non-Spam

**Lets get to it!**


Starting with imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn


In [3]:
df = pd.read_csv('http://bit.ly/SpamCollectionDataset', sep='\t',  header = None, names = ['label', 'message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


We have encountered this dataset before, well start with the preprocessing

In [4]:
# Converting the labels from strings to binary values for our classifier
# 
df['label'] = df.label.map({'ham': 0, 'spam': 1})

In [5]:
# Converting all characters in the message to lower case
# 
df['message'] = df.message.map(lambda x: x.lower())

# Removing any punctuation
# 
df['message'] = df.message.str.replace('[^\w\s]', '')

In [6]:
# Tokenizing the messages into into single words using nltk
import nltk
nltk.download('punkt')

df['message'] = df['message'].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
#We will perform some word stemming
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
 
df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df['message'] = df['message'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message'])

In [9]:
# We could leave it as the simple word-count per message, but it is better to use Term Frequency Inverse Document Frequency, more known as tf-idf
# 
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)


We can do any visualizations on this dataset

# Modeling

In [11]:
from sklearn.model_selection import train_test_split
#Starting at 80/20
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.2, random_state=69)

In [12]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

In [13]:
predicted = model.predict(X_test)
print(np.mean(predicted == y_test))
#very high level of accuracy

0.9479820627802691


Attempting a different version of split


In [14]:
#This is a 70/30 split
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.3, random_state=69)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)
print(np.mean(predicted == y_test))
#slightly higher level of accuracy

0.9485645933014354


In [15]:
#This is a 60/40 split
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.4, random_state=69)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)
print(np.mean(predicted == y_test))
#slightly lower level of accuracy

0.9407806191117093


# Optimization of the model

In [18]:
#We will attempt a smoothing technique 
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.2, random_state=69)
model2 = MultinomialNB(alpha=3).fit(X_train, y_train)
predicted = model2.predict(X_test)
print(np.mean(predicted == y_test))

#This brought down the accuracy 

0.895067264573991


In [21]:
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.2, random_state=69)
model2 = MultinomialNB(alpha=0).fit(X_train, y_train)
predicted = model2.predict(X_test)
print(np.mean(predicted == y_test))

#No smoothing brings back an accuracy of 98%!!!

0.9641255605381166


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


In [22]:
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.2, random_state=69)
model2 = MultinomialNB(alpha=0,fit_prior= False).fit(X_train, y_train)
predicted = model2.predict(X_test)
print(np.mean(predicted == y_test))


0.9641255605381166


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


# **The model is performing extremely well at a 96% accuracy which is acceptable for now**